In [8]:
import numpy as np
import sys 
sys.path.append('/home1/efeghhi/ripple_memory/analysis_code/')
from load_data import *
from analyze_data import *
sys.path.append('/home1/efeghhi/ripple_memory/analysis_code/pac_analyses/')
from load_data_numpy import load_data_np
from comodulogram import remove_session_string, get_filtered_signal

import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import decimate, resample

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert
from scipy.signal import welch
from scipy.signal import spectrogram

In [9]:
run_mode = 1

if run_mode == 1:
    encoding_mode = 1
    saveName = 'encoding_'
    
if run_mode == 2:
    encoding_mode = 0
    saveName = 'recall_'
    
dd_trials = load_data_np(encoding_mode)

# relative to word onset 
start_roi = 300
end_roi = 1300
# each trial consists of 5 sec of raw data, which starts
# 1.7 before word onset and ends 3.3 sec after word onset 
# the data is sampled at 500 Hz
start_time = -1700
end_time = 3300

sr_factor = 2

# convert to indices based on start time and sampling rate factor
start_idx = int((start_roi - start_time)/sr_factor)
end_idx = int((end_roi-start_time)/sr_factor)

print(start_idx, end_idx)

raw_data = dd_trials['raw']


subj_elec_labels = np.array([remove_session_string(x) for x in dd_trials['elec_labels']])


Loading data
Number of trials in ca1: 55356
order: C
1000 1500


In [10]:
raw_data.shape

(55356, 2500)

In [3]:
clust_idxs = np.argwhere(dd_trials['clust_int']==1).squeeze()

In [5]:
unq, counts = np.unique(subj_elec_labels, return_counts=True)

In [7]:
sub_elecs = np.unique(subj_elec_labels)
i = 1
se_idxs = np.argwhere(subj_elec_labels == sub_elecs[i]).squeeze()
clust_se_idxs = np.intersect1d(se_idxs, clust_idxs)
raw_data_se = raw_data[se_idxs]
window_len = raw_data_se.shape[1]
raw_data_se_roi = raw_data_se[:, 500:-500].squeeze()